# Handling Missing Values in Data

Missing data is one of the most common problems in real-world datasets. In this notebook, we'll learn different strategies for dealing with missing values.

## Learning Objectives

By the end of this notebook, you will be able to:
1. Identify missing values in a DataFrame
2. Decide when to delete vs. impute missing values
3. Apply different imputation strategies (mean, median, forward/backward fill)
4. Choose the appropriate strategy based on your data

---

In [ ]:
# Import libraries
import pandas as pd
import numpy as np

print('Libraries loaded!')

---

## Part 1: Identifying Missing Values

Let's create a small dataset with some missing values to practice with.

In [ ]:
# Create a sample dataset with missing values
data = {
    'patient_id': ['P001', 'P002', 'P003', 'P004', 'P005', 'P006'],
    'age': [45, 32, np.nan, 28, 51, 39],
    'weight_kg': [70.5, 65.2, 78.0, np.nan, 82.3, 68.9],
    'blood_pressure': [120, np.nan, 135, 118, np.nan, 122],
    'cholesterol': [200, 185, np.nan, 195, 210, 178]
}

df = pd.DataFrame(data)
print('Patient data:')
print(df)

### Detecting Missing Values

Use `.isnull()` to find missing values (returns True/False for each cell).

Use `.isnull().sum()` to count missing values per column.

In [ ]:
# Check which values are missing
print('Missing values (True = missing):')
print(df.isnull())

print('\nCount of missing values per column:')
print(df.isnull().sum())

### 📝 Practice Question 1

**Task:** Calculate the **percentage** of missing values for each column.

**Hint:** Divide the count of missing values by the total number of rows, then multiply by 100.

In [ ]:
# YOUR CODE HERE
# Calculate percentage of missing values per column


---

## Part 2: Strategy 1 - Deleting Missing Values

### When to Delete?
- **Delete rows:** When only a few rows have missing data
- **Delete columns:** When a column has mostly missing values (e.g., >50% missing)

### Deleting Rows with Missing Values

In [ ]:
# Drop any row with at least one missing value
df_no_missing = df.dropna()

print('Original data had', len(df), 'rows')
print('After dropping rows with missing values:', len(df_no_missing), 'rows')
print('\nRemaining data:')
print(df_no_missing)

⚠️ **Warning:** We lost 5 out of 6 patients! This is too much data loss.

### Deleting Columns with Too Many Missing Values

Let's create a dataset where one column has mostly missing values.

In [ ]:
# Create data with one column having many missing values
data2 = {
    'patient_id': ['P001', 'P002', 'P003', 'P004', 'P005', 'P006'],
    'age': [45, 32, 38, 28, 51, 39],
    'weight_kg': [70.5, 65.2, 78.0, 72.1, 82.3, 68.9],
    'rare_test': [np.nan, np.nan, 5.2, np.nan, np.nan, np.nan]  # 83% missing!
}

df2 = pd.DataFrame(data2)
print('Dataset with one column having many missing values:')
print(df2)

print('\nMissing value percentages:')
print((df2.isnull().sum() / len(df2) * 100).round(1))

In [ ]:
# Drop column with >50% missing values
threshold = 0.5  # 50%
missing_pct = df2.isnull().sum() / len(df2)
cols_to_keep = missing_pct[missing_pct <= threshold].index

df2_clean = df2[cols_to_keep]

print('Columns kept:', list(df2_clean.columns))
print('\nCleaned data:')
print(df2_clean)

### 📝 Practice Question 2

**Task:** Using the original `df`, drop only the rows where **age** is missing (keep rows with other missing values).

**Hint:** Use `dropna(subset=['column_name'])`

In [ ]:
# YOUR CODE HERE
# Drop rows where age is missing


---

## Part 3: Strategy 2 - Imputing with Mean or Median

### When to Use?
- **Mean:** When data is roughly normally distributed (no extreme outliers)
- **Median:** When data has outliers or is skewed

### Imputing with Mean

In [ ]:
# Make a copy to preserve original
df_mean = df.copy()

# Fill missing age values with mean age
age_mean = df_mean['age'].mean()
df_mean['age'] = df_mean['age'].fillna(age_mean)

print(f'Mean age: {age_mean:.1f}')
print('\nData after filling age with mean:')
print(df_mean[['patient_id', 'age']])

### Imputing with Median

In [ ]:
# Make a copy to preserve original
df_median = df.copy()

# Fill missing blood_pressure with median
bp_median = df_median['blood_pressure'].median()
df_median['blood_pressure'] = df_median['blood_pressure'].fillna(bp_median)

print(f'Median blood pressure: {bp_median:.1f}')
print('\nData after filling blood pressure with median:')
print(df_median[['patient_id', 'blood_pressure']])

### Filling Multiple Columns at Once

In [ ]:
# Fill all numeric columns with their median
df_all_median = df.copy()

numeric_cols = ['age', 'weight_kg', 'blood_pressure', 'cholesterol']

for col in numeric_cols:
    median_val = df_all_median[col].median()
    df_all_median[col] = df_all_median[col].fillna(median_val)
    print(f'{col}: filled with {median_val:.1f}')

print('\nData after filling all columns with median:')
print(df_all_median)

### 📝 Practice Question 3

**Task:** Fill the missing values in the `weight_kg` column with the **mean** weight.

Print the DataFrame before and after to see the change.

In [ ]:
# YOUR CODE HERE
# Fill missing weight_kg with mean


---

## Part 4: Strategy 3 - Forward Fill and Backward Fill

### When to Use?
- **Time series data** (e.g., daily temperature, stock prices)
- **Sequential measurements** (e.g., patient vitals over time)

### Forward Fill (ffill)
Uses the **previous** value to fill missing values.

In [ ]:
# Create time series data (daily temperature readings)
temp_data = {
    'day': ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
    'temperature': [22.5, 23.1, np.nan, np.nan, 21.8, 22.3, np.nan]
}

df_temp = pd.DataFrame(temp_data)
print('Temperature data with missing values:')
print(df_temp)

In [ ]:
# Forward fill - use previous value
df_ffill = df_temp.copy()
df_ffill['temperature'] = df_ffill['temperature'].ffill()

print('After forward fill (use previous day):')
print(df_ffill)

### Backward Fill (bfill)
Uses the **next** value to fill missing values.

In [ ]:
# Backward fill - use next value
df_bfill = df_temp.copy()
df_bfill['temperature'] = df_bfill['temperature'].bfill()

print('After backward fill (use next day):')
print(df_bfill)

### 📝 Practice Question 4

**Task:** Create a DataFrame with patient heart rate measurements over 5 time points. Include 2-3 missing values.

Then fill the missing values using **forward fill**.

**Hint:** Use the temperature example as a template.

In [ ]:
# YOUR CODE HERE
# Create heart rate data and apply forward fill


---

## Part 5: Choosing the Right Strategy

Here's a decision guide:

| Situation | Strategy | Method |
|-----------|----------|--------|
| < 5% rows with missing values | Delete rows | `dropna()` |
| Column has >50% missing | Delete column | `drop(columns=[...])` |
| Normal distribution, few outliers | Fill with mean | `.fillna(mean)` |
| Skewed data or outliers | Fill with median | `.fillna(median)` |
| Time series / sequential data | Forward/backward fill | `.ffill()` or `.bfill()` |

### Real-World Example

In [ ]:
# Gene expression data (typical in biology)
gene_data = {
    'sample_id': ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8'],
    'BRCA1': [5.2, 5.8, np.nan, 5.5, 5.1, 5.9, 5.3, 5.7],
    'TP53': [7.1, np.nan, 7.3, 7.0, 7.2, 7.4, 7.1, 7.3],
    'rare_gene': [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 2.1, np.nan]
}

df_genes = pd.DataFrame(gene_data)

print('Gene expression data:')
print(df_genes)
print('\nMissing percentages:')
print((df_genes.isnull().sum() / len(df_genes) * 100).round(1))

In [ ]:
# Step 1: Drop column with >50% missing
df_genes_clean = df_genes.drop(columns=['rare_gene'])
print('Step 1: Dropped rare_gene column')

# Step 2: Fill remaining missing values with median (gene expression often skewed)
df_genes_clean['BRCA1'] = df_genes_clean['BRCA1'].fillna(df_genes_clean['BRCA1'].median())
df_genes_clean['TP53'] = df_genes_clean['TP53'].fillna(df_genes_clean['TP53'].median())
print('Step 2: Filled BRCA1 and TP53 with median')

print('\nFinal cleaned data:')
print(df_genes_clean)
print('\nMissing values remaining:', df_genes_clean.isnull().sum().sum())

### 📝 Practice Question 5 (Challenge)

**Task:** You have patient data with the following missing value percentages:
- `patient_id`: 0%
- `age`: 5%
- `weight`: 8%
- `experimental_marker`: 85%
- `daily_steps` (time series): 12%

**Decide and implement:**
1. Which column(s) should you delete?
2. Which column(s) should use mean/median?
3. Which column(s) should use forward fill?

Create a small example dataset and apply your strategy.

In [ ]:
# YOUR CODE HERE
# Create the dataset and apply appropriate strategies


---

## Summary

### Key Takeaways:

1. **Always check for missing values first:** `df.isnull().sum()`

2. **Deletion strategies:**
   - `dropna()` - remove rows with any missing values
   - `dropna(subset=['col'])` - remove rows with missing in specific column
   - Drop columns with >50% missing

3. **Imputation strategies:**
   - `fillna(mean)` - for normal distributions
   - `fillna(median)` - for skewed data or outliers
   - `ffill()` - for time series (use previous value)
   - `bfill()` - for time series (use next value)

4. **No perfect solution:** The best strategy depends on:
   - How much data is missing
   - Type of data (numerical, categorical, time series)
   - Your analysis goals

### Next Steps:
- Practice with your own datasets
- Always document which strategy you used and why
- Consider the impact on your downstream analysis